# Analysis

This file contains the public data analysis for "Characterizing the US Research Computing and Data (RCD) Workforce."

In [1]:
library(tidyverse)
library(janitor)

── Attaching packages ────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




## Data

In [2]:
all_freq <- read_csv("data_to_share/var_frequencies_all_respondents.csv")
institutions <- read_csv("data_to_share/institutions_all.csv")
all_edu_work <- read_csv("data_to_share/education_work_all_respondents.csv")
rcd_edu_work <- read_csv("data_to_share/education_work_rcd_respondents.csv")
gender <- read_csv("data_to_share/gender_counts.csv")
ethnicity <- read_csv("data_to_share/ethnicity_counts.csv")
jobs <- read_csv("data_to_share/job_titles.csv")
facings <- read_csv("data_to_share/facings.csv")
satisfaction <- read_csv("data_to_share/satisfaction_rcd_respondents.csv")
inclusion <- read_csv("data_to_share/inclusion.csv")

Rows: 150 Columns: 4
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): variable, category
dbl (2): n, prop

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 563 Columns: 6
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): employer_type, institution_type, institution_code, institution_comb...
lgl (1): academic_institution

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 563 Columns: 20
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): education, education_related
dbl  (3): experience_rcd, experience_wo

## 3: SURVEYING THE WORKFORCE

Institution Counts

In [6]:
length(unique(institutions$institution_code))

[1] 175

In [7]:
institutions %>%
  distinct(institution_code, .keep_all=TRUE) %>%
  tabyl(institution_type)

,institution_type,n,percent,valid_percent
,<chr>,<int>,<dbl>,<dbl>
1,Academic Computing Center,5,0.028571429,0.02873563
2,Company,8,0.045714286,0.04597701
3,Government,5,0.028571429,0.02873563
4,Medical,10,0.057142857,0.05747126
5,Other,7,0.040000000,0.04022989
6,Other Academic,18,0.102857143,0.10344828
7,R1,91,0.520000000,0.52298851
8,R2,30,0.171428571,0.17241379
9,NA,1,0.005714286,NA


In [8]:
institutions %>%
  filter(!is.na(institution_code)) %>%
  count(institution_code) %>%
  summarize(median(n),
            max(n))

median(n),max(n)
<dbl>,<int>
2,26


R1/R2

In [9]:
institutions %>%
  filter(academic_institution) %>%
  summarize(R1_respondents = sum(institution_type == "R1")/n(),
            R1_inst = n_distinct(institution_code[institution_type == "R1"])/n_distinct(institution_code))

R1_respondents,R1_inst
<dbl>,<dbl>
0.8163717,0.6319444


Respondents

In [10]:
filter(all_freq, variable == "rcd_employed")

variable,category,n,prop
<chr>,<chr>,<dbl>,<dbl>
rcd_employed,No/No Answer,24,0.04262877
rcd_employed,Yes,539,0.95737123


In [11]:
sum(rcd_edu_work$academic_institution, na.rm=TRUE)/sum(!is.na(rcd_edu_work$academic_institution))

[1] 0.8607843

## 4.1 Demographics

### Age

In [12]:
filter(all_freq, variable=="age")

variable,category,n,prop
<chr>,<chr>,<dbl>,<dbl>
age,20 - 24,11,0.01953819
age,25 - 34,96,0.17051510
age,35 - 44,167,0.29662522
age,45 - 54,171,0.30373002
age,55 - 64,99,0.17584369
age,65 - 74,19,0.03374778


### Race/Ethnicity

In [14]:
filter(all_freq, variable=="ethnicity1")

variable,category,n,prop
<chr>,<chr>,<dbl>,<dbl>
ethnicity1,Asian,59,0.10479574
ethnicity1,Black or African American,10,0.01776199
ethnicity1,Multiple/Native Hawaiian or Pacific Islander/Jewish/Middle Eastern/Mediterranean/Other Unspecified,15,0.02664298
ethnicity1,White (Hispanic),9,0.01598579
ethnicity1,White (Not Hispanic),435,0.77264654
ethnicity1,NA,35,0.06216696


In [15]:
filter(all_freq, variable=="hispanic")

variable,category,n,prop
<chr>,<chr>,<dbl>,<dbl>
hispanic,FALSE,516,0.91651865
hispanic,TRUE,13,0.02309059
hispanic,NA,34,0.06039076
